# 10 - Prática: Lidando com Dados do Mundo Real (II) (LEIA A DESCRIÇÃO COMPLETA)

ODS: O INSTRUTOR RETIROU OS ARQUIVOS DE CÓDIGO DO CURSO, MAS ANEXAMOS OS MESMOS PARA VOCÊ AQUI NO CARD, BAIXE E USE DURANTE O CURSO.

Prazo para entrega: 10 dias. Criar data de entrega utilizando a data de cópia do cartão e o prazo estabelecido para esta tarefa.

Entrega a Ser Feita: Relatório com descrição dos principais conhecimentos adquiridos, anexar o arquivo ao seu cartão.

Seções dentro de um minicurso com conceitos importantes sobre como entender e manipular dados do mundo real ao lidar com projetos de Machine Learning e Inteligência Artificial.

O vídeo é inglês e interessante seria o acompanhamento com legendas em inglês, mas se sentir dificuldades pode configurar para que o youtube gere legendas em português sendo a qualidade destas muito boa.

Você irá acessar a nossa conta da Udemy e ir o seguinte curso: Machine Learning, Data Science and Generative AI with Python. Dentro desse curso faça apenas as seções que estão descritas no checklist abaixo.

KNN é um conceito simples: defina alguma métrica de distância entre os itens em seu conjunto de dados e encontre os K itens mais próximos. Você pode então usar esses itens para prever alguma propriedade de um item de teste, fazendo com que eles de alguma forma "votem" nele. Como exemplo, vejamos os dados do MovieLens. Tentaremos adivinhar a classificação de um filme observando os 10 filmes mais próximos dele em termos de gênero e popularidade. Para começar, carregaremos todas as classificações do conjunto de dados em um DataFrame do Pandas:

In [1]:
import pandas as pd 

r_cols = ['user_id','movie_id','rating']
ratings = pd.read_csv('ml-100k/u.data', sep='\t',names = r_cols, usecols = range(3))
ratings.head()


,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


Agora, agruparemos tudo por ID do filme e calcularemos o número total de avaliações (a popularidade de cada filme) e a classificação média de cada filme:

In [2]:
import numpy as np

movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})
movieProperties.head()

C:\Users\jonin\AppData\Local\Temp\ipykernel_18060\579369499.py:3: FutureWarning: The provided callable <function mean at 0x000001A2136AD9E0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})


rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

O número bruto de classificações não é muito útil para calcular distâncias entre filmes, por isso criaremos um novo DataFrame que contém o número normalizado de classificações. Portanto, um valor 0 significa que ninguém o avaliou e um valor 1 significa que é o filme mais popular que existe.

In [3]:
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


Agora, vamos obter as informações do gênero no arquivo u.item. A forma como isto funciona é que existem 19 campos, cada um correspondendo a um género específico - um valor de '0' significa que não está nesse género, e '1' significa que está nesse género. Um filme pode ter mais de um gênero associado a ele. Enquanto estamos nisso, reuniremos tudo em um grande dicionário Python chamado movieDict. Cada entrada conterá o nome do filme, a lista de valores de gênero, a pontuação de popularidade normalizada e a classificação média de cada filme:

In [4]:
movieDict = {}
with open('ml-100k/u.item', encoding="ISO-8859-1") as f:
    temp = ''
    for line in f:
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        name = fields[1]
        genres= fields[5:25]
        genres = map(int, genres)
        movieDict[movieID] = (name, np.array(list(genres)), movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))

Por exemplo, aqui está o registro que obtivemos para o ID do filme 1, "Toy Story":

In [5]:
movieDict[2]

('GoldenEye (1995)',
 array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
 0.22298456260720412,
 3.2061068702290076)

In [6]:
movieDict[1] 

('Toy Story (1995)',
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0.7735849056603774,
 3.8783185840707963)

Agora vamos definir uma função que calcula a "distância" entre dois filmes com base na semelhança entre seus gêneros e na popularidade. Só para ter certeza de que funciona, calcularemos a distância entre os IDs 2 e 4 do filme:

In [7]:
from scipy import spatial

def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance
    
ComputeDistance(movieDict[2], movieDict[4])



0.8004574042309892

Lembre-se de que quanto maior a distância, menos semelhantes serão os filmes. Vamos verificar o que realmente são os filmes 2 e 4 - e confirmar que não são tão parecidos:

In [8]:
print(movieDict[2])
print(movieDict[4])

('GoldenEye (1995)', array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.22298456260720412, 3.2061068702290076)
('Get Shorty (1995)', array([0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.3567753001715266, 3.550239234449761)


Agora, só precisamos de um pequeno código para calcular a distância entre um determinado filme de teste (Toy Story, neste exemplo) e todos os filmes em nosso conjunto de dados. Ao classificá-los por distância e imprimir os K vizinhos mais próximos:

In [9]:
import operator

def getNeighbors(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist  = ComputeDistance(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for i in range(K):
        neighbors.append(distances[i][0])
    return neighbors

k = 10
avgRating = 0
neighbors = getNeighbors(1, k)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print(movieDict[neighbor][0] + "" + str(movieDict[neighbor][3]))
    
avgRating /= k 
    

Liar Liar (1997)3.156701030927835
Aladdin (1992)3.8127853881278537
Willy Wonka and the Chocolate Factory (1971)3.6319018404907975
Monty Python and the Holy Grail (1974)4.0664556962025316
Full Monty, The (1997)3.926984126984127
George of the Jungle (1997)2.685185185185185
Beavis and Butt-head Do America (1996)2.7884615384615383
Birdcage, The (1996)3.4436860068259385
Home Alone (1990)3.0875912408759123
Aladdin and the King of Thieves (1996)2.8461538461538463


Enquanto estávamos nisso, calculamos a classificação média dos 10 vizinhos mais próximos de Toy Story:

In [10]:
avgRating

3.3445905900235564

Como isso se compara à classificação média real de Toy Story?

In [11]:
movieDict[1]

('Toy Story (1995)',
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0.7735849056603774,
 3.8783185840707963)

Nossa escolha de 10 para K foi arbitrária – que efeito os diferentes valores de K têm nos resultados? Nossa métrica de distância também era um tanto arbitrária - apenas pegamos a distância cosseno entre os gêneros e a adicionamos à diferença entre as pontuações de popularidade normalizadas. Você pode melhorar isso?

In [12]:
import numpy as np
import operator
from scipy import spatial

# Dicionário para armazenar os filmes
movieDict = {}

# Abrindo o arquivo e populando o dicionário
with open('ml-100k/u.item', encoding="ISO-8859-1") as f:
    for line in f:
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:25]
        genres = map(int, genres)
        movieDict[movieID] = (name, np.array(list(genres)), 
                              movieNormalizedNumRatings.loc[movieID].get('size'), 
                              movieProperties.loc[movieID].rating.get('mean'))

In [23]:
# Função para calcular os vizinhos mais próximos
def getNeighbors(movieID, K):
    distances = []
    for movie in movieDict:
        if movie != movieID:
             
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    # Ordenar pela distância
    distances.sort(key=operator.itemgetter(1))
    neighbors = [distances[i][0] for i in range(K)]
    return neighbors

In [24]:
for movieID in movieDict:
    if movieID != 1:
        print(f"Distance between movie 1 and {movieID}: {ComputeDistance(movieDict[1], movieDict[movieID])}")


Distance between movie 1 and 2: 0.8651801029159519
Distance between movie 1 and 3: 0.8862778730703259
Distance between movie 1 and 4: 0.5917095483133219
Distance between movie 1 and 5: 0.888336192109777
Distance between movie 1 and 6: 0.9192109777015437
Distance between movie 1 and 7: 0.7308747855917667
Distance between movie 1 and 8: 0.3532304173813608
Distance between movie 1 and 9: 0.7787307032590051
Distance between movie 1 and 10: 0.8867924528301887
Distance between movie 1 and 11: 0.8111492281303602
Distance between movie 1 and 12: 0.7951972555746141
Distance between movie 1 and 13: 0.43376218721048665
Distance between movie 1 and 14: 0.8384219554030874
Distance between movie 1 and 15: 0.7818181818181817
Distance between movie 1 and 16: 0.6267476374986234
Distance between movie 1 and 17: 0.7045094907275875
Distance between movie 1 and 18: 0.9274442538593481
Distance between movie 1 and 19: 0.8970840480274442
Distance between movie 1 and 20: 0.8955403087478558
Distance between mov

In [25]:
# Função para calcular a distância entre dois filmes
def ComputeDistance(a, b, weight_genre=0.7, weight_popularity=0.3):
    genresA = a[1]
    genresB = b[1]
    
    # Distância cosseno entre os gêneros
    genreDistance = spatial.distance.cosine(genresA, genresB)
    
    # Diferença de popularidade
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    
    # Combinação das distâncias com pesos
    return (weight_genre * genreDistance) + (weight_popularity * popularityDistance)

In [15]:
# Exemplo de chamada para calcular a distância entre dois filmes
ComputeDistance(movieDict[2], movieDict[4])

0.5068038879359634

In [16]:
# Função para calcular a média das avaliações dos K vizinhos mais próximos
def getAverageRating(movieID, K):
    neighbors = getNeighbors(movieID, K)
    avgRating = sum([movieDict[neighbor][3] for neighbor in neighbors]) / K
    return avgRating

In [17]:
# Exemplo de uso para encontrar os 100 vizinhos mais próximos e a média de suas avaliações
k = 10
avgRating = getAverageRating(1, k)


In [18]:

# Imprime os vizinhos mais próximos e suas avaliações
neighbors = getNeighbors(1, k)
for neighbor in neighbors:
    print(f"{movieDict[neighbor][0]}: {movieDict[neighbor][3]}")


Aladdin (1992): 3.8127853881278537
Aladdin and the King of Thieves (1996): 2.8461538461538463
George of the Jungle (1997): 2.685185185185185
Beavis and Butt-head Do America (1996): 2.7884615384615383
Home Alone (1990): 3.0875912408759123
Jungle2Jungle (1997): 2.4393939393939394
Willy Wonka and the Chocolate Factory (1971): 3.6319018404907975
Wrong Trousers, The (1993): 4.466101694915254
101 Dalmatians (1996): 2.908256880733945
Pinocchio (1940): 3.6732673267326734


In [19]:

# Exemplo de uso para testar diferentes valores de K
for k_value in [10, 50, 100]:
    avgRating = getAverageRating(1, k_value)
    print(f"Average rating for K={k_value}: {avgRating}")

Average rating for K=10: 3.233909888107094
Average rating for K=50: 3.0838838256764904
Average rating for K=100: 3.1395954722661616
